In [2]:
from langchain_ollama import OllamaLLM
import pandas as pd
import ast
import re
from difflib import SequenceMatcher

def askOllama(prompt):
    result = model.invoke(input=prompt)
    return result

def cleanOllamaOutput(output):
    pattern = r"\[.*?\]"
    
    matches = re.findall(pattern, output, re.DOTALL)
    a = matches[0].replace("\n", "")
    escaped_string = re.sub(r"(?<=\w)'(?=\w)", r"\'", a)
    return ast.literal_eval(escaped_string.replace("\\n", "").replace("\\\\", "\\"))

model = OllamaLLM(model="llama3.2")

annotatedDataset = pd.read_csv("/Users/sagewong/git/StigmatizingLanguageProject/FinalFinalAnnnotatedData.csv")


In [6]:
true_positive = 0
false_positive = 0
true_negative = 0
false_negative = 0
for index in range(annotatedDataset.shape[0]):
    print("INDEX: " + str(index))
    clinicalNote = annotatedDataset.iloc[index]['Completion']

    answer = ast.literal_eval(annotatedDataset.iloc[index]['annotated'])


    prompt = """You are a professional linguist researcher who is trying to identify stigmatizing language in clinical notes. Given this clinical note, return to me in a python-type list all forms of stigmatizing language (e.g. noncompliant, nonadherent, etc...). Do not include any descriptions or explanations. Also do not rewrite the stigmatizing language in your own words. Here's an example of a note: **Admission date:** 09/22/2023

**Discharge date:** 09/28/2023

**Date of Birth:** 07/04/1967

**Sex:** F

**Service:** MEDICINE

**Allergies:** No known allergies

**Attending:** Dr. Jameson

**Chief Complaint:** Persistent abdominal pain

**Major Surgical or Invasive Procedure:** None during this admission

**History of Present Illness:**
Mrs. Ernser presented with a week-long history of worsening abdominal pain, not alleviated by OTC medications. Reports issues began following a meal from a fast-food restaurant, suspicious of food poisoning but has been resistant to follow recommended dietary adjustments.

**Past Medical History:**
- History of Type 2 Diabetes
- Hypertension
- Previous incidents of medication nonadherence

**Social History:**
Noted to be challenging in her social interactions, often declining support offered by community health workers. Mentioned that she spends time around local fast-food outlets like McDonald’s, despite dietary advice.

**Family History:**
No known family history of similar illness.

**Physical Exam:**
VS: Temp 98.6 F, BP 140/90, Pulse 88, RR 16 
General: Appears stated age 
HEENT: Unremarkable 
Neck: Supple
CV: RRR, no murmurs 
Lungs: Clear to auscultation bilaterally 
Abdomen: Tender in the upper quadrant 
GU: Normal exam 
Ext: No edema 
Neuro: Alert and oriented, but somewhat noncompliant during cognitive tests 

**Pertinent Results:**
09/22/2023 - Labs indicated elevated blood sugar levels, patient declined further diagnostic exams initially.

**Brief Hospital Course:**
Mrs. Ernser’s hospital stay was marked by her reluctance to participate in several recommended tests, terming her a difficult patient in managing her care plan. Despite this, a conservative approach to manage her abdominal pain was adopted. Noted issues with compliance regarding dietary recommendations were evident, impacting her condition.

**Medications on Admission:**
1. Metformin
2. Lisinopril
3. Aspirin
4. Simvastatin
5. Insulin regimen (patient has been inconsistent)
6. Multivitamin
7. Ibuprofen as needed

**Discharge medications:**
1. Metformin (continuation)
2. Lisinopril (dose adjustment)
3. Aspirin
4. Simvastatin
5. Adjusted Insulin regimen (stress on adherence)
6. Multivitamin
7. Antibiotic for suspected infection
8. Follow-up with dietary supplements

**Discharge Disposition:** home

**Discharge Diagnosis:** Gastroenteritis likely due to foodborne illness, poorly controlled diabetes

**Discharge Condition:**
Mental Status: Alert
Activity Status: Limited due to pain

**Discharge Instructions:**
Mrs. Ernser, it’s crucial to adhere to the insulin regimen and dietary modifications to manage your diabetes and prevent future health issues. Please take all medications as prescribed and maintain a balanced diet, avoiding fast food, to improve your condition. A return to the hospital could be necessary if symptoms persist or worsen. Keep track of your blood sugar levels daily. Schedule a follow-up appointment with your primary care provider and a diabetes specialist within two weeks to reassess your condition and treatment plan. Focus on incorporating light exercise into your daily routine as tolerated. For any questions or in case of an emergency, do not hesitate to contact the hospital or your primary care doctor.

**Followup Instructions:**
- Primary care provider in 2 weeks
- Diabetes specialist within 1 month""" + "And here's an example of the stigmatizing language extracted from it: " + """[
    'challenging in her social interactions',
    'resistant to follow recommended dietary adjustments',
    'difficult patient in managing her care plan',
    'noncompliant during cognitive tests',
    'inconsistent insulin regimen',
    'poorly controlled diabetes'
]""" + "And here's the actual note you will have to analyze, and make sure you output the list of stigmatizing words in JSON output: " + clinicalNote

    rawOutput = askOllama(prompt)
    cleanedOutput = cleanOllamaOutput(rawOutput)
    print(cleanedOutput)
    print(answer)

    while True:
        try:
            matching = cleanOllamaOutput(askOllama("Return to me only a list of elements which are referring to the same thing in these two lists in JSON format. Here are the two lists: " + str(cleanedOutput) + ", " + str(answer)))
            break
        except:
            print()

    print(matching)

    true_positive += len(matching)
    false_positive += len([i for i in cleanedOutput if i not in matching])
    true_negative += clinicalNote.count(" ")/3 - len(matching) + len([i for i in answer if answer not in matching])*3
    false_negative += len([i for i in answer if answer not in matching])

    tempTruePositive = len(matching)
    tempFalsePositive = len([i for i in cleanedOutput if i not in matching])
    tempTrueNegative = clinicalNote.count(" ")/3 - len(matching) + len([i for i in answer if answer not in matching])*3
    tempFalseNegative = len([i for i in answer if answer not in matching])
    tempPrecision = tempTruePositive/(tempTruePositive + tempFalsePositive)
    tempRecall = tempTruePositive/(tempTruePositive + tempFalseNegative)
    try:    
        tempf1score = (2*tempPrecision*tempRecall)/(tempPrecision+tempRecall)
        print("SINGLE NOTE F1 SCORE: " + str(tempf1score))
    except:
        print("SINGLE NOTE F1 SCORE: 0")
    precision = true_positive/(true_positive + false_positive)
    recall = true_positive/(true_positive + false_negative)

    f1_score = (2*precision*recall)/(precision+recall)
    print("TOTAL F1 SCORE: " + str(f1_score))

precision = true_positive/(true_positive + false_positive)
recall = true_positive/(true_positive + false_negative)

f1_score = (2*precision*recall)/(precision+recall)
print(precision)
print(recall)
print(f1_score)

INDEX: 0
['challenging', 'noncompliant', 'difficult patient', 'uncooperative behavior', 'prefers junk food', 'cursing at nurses', 'poor dietary habits', 'unreliable historical recounting', 'refused several exams', 'resistant to modifying his diet', 'persisted with cursing at staff']
['challenging patient', 'uncooperative behavior', 'noncompliant', 'citing his preferences for junk food', 'frequent cursing at nurses', 'difficult patient', 'challenging interactions', 'unreliable historical recounting', 'refused several exams', 'resistant to modifying his diet', 'complicating our attempts to provide care', 'significant barrier to his care', 'noncompliance has been a significant barrier to his care', "Mohammad's noncompliance has been a significant barrier to his care"]
[{'element': 'challenging patient', 'count': 2}, {'element': 'uncooperative behavior', 'count': 2}, {'element': 'noncompliant', 'count': 3}, {'element': 'cursing at nurses', 'count': 2}, {'element': 'difficult patient', 'cou